# Système de gestion d’emprunt dans une librairie

In [33]:
# Importation des librairies
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import max

In [34]:
# Session spark
spark = SparkSession.builder.master('local').appName("Library").getOrCreate()

### Création des tables de données

#### *La table Author :*
représente une table d'auteurs. Chaque ligne contient le nom et l'identifiant d'un auteur

In [35]:
# Author table
l1 = [("07890", 'Jean Paul Sartre'), ("05678", 'Pierre de Ronsard')]
rdd1 = spark.sparkContext.parallelize(l1)
Author = rdd1.toDF(['aid','name'])
Author.createOrReplaceTempView("Author")
Author.show()

+-----+-----------------+
|  aid|             name|
+-----+-----------------+
|07890| Jean Paul Sartre|
|05678|Pierre de Ronsard|
+-----+-----------------+



#### *La table book :*
représente une table de livres. Chaque ligne est un livre décrit par son identifiant, son titre et sa catégorie (roman, science-fiction, musique, etc.).

In [6]:
# Book Table
l2 = [('0001',"L'existentialisme est un humanisme","Philosophie"),
      ('0002',"Huis clos. Suivi de Les Mouches", "Philosophie"),
      ('0003',"Mignonne allons voir si la rose", "Poeme"),
      ('0004',"Les Amours", "Poeme")]
rdd2 = spark.sparkContext.parallelize(l2) 
book = rdd2.toDF(['bid','title','category'])
book.createOrReplaceTempView("book")
book.show(truncate=False)

+----+----------------------------------+-----------+
|bid |title                             |category   |
+----+----------------------------------+-----------+
|0001|L'existentialisme est un humanisme|Philosophie|
|0002|Huis clos. Suivi de Les Mouches   |Philosophie|
|0003|Mignonne allons voir si la rose   |Poeme      |
|0004|Les Amours                        |Poeme      |
+----+----------------------------------+-----------+



#### *La table Student :*
représente une table d'étudiants. Chaque ligne est un étudiant décrit par son identifiant, son nom et son département (informatique, mécanique...).

In [36]:
# Student table
l3 = [('S15',"toto","Maths"),
      ('S16',"popo", "Eco"),
      ('S17',"fofo", "Mécanique")]
rdd3 = spark.sparkContext.parallelize(l3) 
Student = rdd3.toDF(['sid','sname','dept'])
Student.createOrReplaceTempView("Student")
Student.show()

+---+-----+---------+
|sid|sname|     dept|
+---+-----+---------+
|S15| toto|    Maths|
|S16| popo|      Eco|
|S17| fofo|Mécanique|
+---+-----+---------+



#### *La table write :*
représente l'association entre les auteurs et les livres. Une ligne de cette table signifie que l'auteur a écrit le livre bid.

In [37]:
# Write table
l4 = [('07890',"0001"),
      ('07890',"0002"),
      ('05678',"0003"),
      ('05678',"0004")]
rdd4 = spark.sparkContext.parallelize(l4) 
write = rdd4.toDF(['aid','bid'])
write.createOrReplaceTempView("write")
write.show()

+-----+----+
|  aid| bid|
+-----+----+
|07890|0001|
|07890|0002|
|05678|0003|
|05678|0004|
+-----+----+



#### *La table borrow :*

 représente les informations de prêt de livre. 
Une ligne de cette table signifie que l'étudiant a emprunté le livre bid  , à la date checkout-time et l'a retourné à la date return-time 

In [55]:
# Borrow table
l5 = [('S15',"0003","02-01-2020","01-02-2020"),
      ('S15',"0002","13-06-2020", None),
      ('S15',"0001","13-06-2020","13-10-2020"),
      ('S16',"0002", "24-01-2020","24-01-2020"),
      ('S17',"0001", "12-04-2020","01-07-2020")]
rdd5 = spark.sparkContext.parallelize(l5) 
borrow = rdd5.toDF(['sid','bid','checkout-time','return-time'])
borrow.createOrReplaceTempView("borrow")
borrow.show()

+---+----+-------------+-----------+
|sid| bid|checkout-time|return-time|
+---+----+-------------+-----------+
|S15|0003|   02-01-2020| 01-02-2020|
|S15|0002|   13-06-2020|       null|
|S15|0001|   13-06-2020| 13-10-2020|
|S16|0002|   24-01-2020| 24-01-2020|
|S17|0001|   12-04-2020| 01-07-2020|
+---+----+-------------+-----------+



### 1- Les titres de tous les livres que l'étudiant sid='S15' a emprunté.

In [39]:
# SQL
spark.sql(''' select title
              from Student as s join borrow as b on s.sid = b.sid
                                   join book as bk on b.bid = bk.bid
              where s.sid = "S15"
              ''').show(truncate=False)

+----------------------------------+
|title                             |
+----------------------------------+
|Huis clos. Suivi de Les Mouches   |
|Mignonne allons voir si la rose   |
|L'existentialisme est un humanisme|
+----------------------------------+



In [40]:
# DSL
Student.join(borrow, on=["sid"])\
     .join(book, on=["bid"])\
     .select("title")\
     .filter(col("sid") == "S15")\
     .show(truncate=False)

+----------------------------------+
|title                             |
+----------------------------------+
|Huis clos. Suivi de Les Mouches   |
|Mignonne allons voir si la rose   |
|L'existentialisme est un humanisme|
+----------------------------------+



### 2- Les titres de tous les livres qui n'ont jamais été empruntés par un étudiant.

In [41]:
# SQL
spark.sql(''' select bid, title
              from book
              EXCEPT
              select distinct(b.bid), title
              from borrow as b join book as bk on b.bid = bk.bid
             
              ''').show()

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



In [43]:
# DSL :
book.join(borrow, on=["bid"], how="left_anti")\
    .select("bid","title")\
    .show()

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



### 3- Tous les étudiants qui ont emprunté le livre bid=’002’

In [44]:
# SQL
spark.sql(''' select sname
              from Student as s join borrow as b on s.sid = b.sid
                                   join book as bk on b.bid = bk.bid
              where b.bid = "0002"
              
              ''').show()

+-----+
|sname|
+-----+
| popo|
| toto|
+-----+



In [45]:
# DSL
Student.join(borrow, on=["sid"])\
     .join(book, on=["bid"])\
     .select("sname")\
     .filter(borrow.bid == "0002")\
     .show()

+-----+
|sname|
+-----+
| popo|
| toto|
+-----+



### 4- Les titres de tous les livres empruntés par les étudiants en Mécanique

In [46]:
# SQL
spark.sql(''' select title
              from Student as s join borrow as b on s.sid = b.sid
                                   join book as bk on b.bid = bk.bid
              where dept = "Mécanique"
              ''').show(truncate=False)

+----------------------------------+
|title                             |
+----------------------------------+
|L'existentialisme est un humanisme|
+----------------------------------+



In [47]:
# DSL
Student.join(borrow, on=["sid"])\
     .join(book, on=["bid"])\
     .select("title")\
     .filter(col("dept") == "Mécanique")\
     .show(truncate=False)

+----------------------------------+
|title                             |
+----------------------------------+
|L'existentialisme est un humanisme|
+----------------------------------+



### 5- Les étudiants qui n’ont jamais emprunté de livre. 

In [48]:
# SQL
spark.sql(''' select sid, sname
              from Student
              EXCEPT
              select distinct(s.sid), sname
              from Student as s join borrow as b on s.sid = b.sid
             
              ''').show()

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



In [50]:
Student.join(borrow, on=["sid"], how="left_anti")\
    .select("sid", "sname")\
    .show()

+---+-----+
|sid|sname|
+---+-----+
+---+-----+



**Result:** Tous les étudiants ont déjà emprunté un livre

### 6- L’auteur qui a écrit le plus de livres.

In [51]:
# SQL
spark.sql(''' select name, count(title) as Nombre_de_livres
              from Author as a join write as w on a.aid = w.aid
                                   join book as bk on w.bid = bk.bid
              group by name
              having Nombre_de_livres = (select max(Nombre_de_livres) from 
                                         (select name, count(title) as Nombre_de_livres
                                          from Author as a join write as w on a.aid = w.aid
                                                             join book as bk on w.bid = bk.bid
                                          group by name))
              ''').show()

+-----------------+----------------+
|             name|Nombre_de_livres|
+-----------------+----------------+
| Jean Paul Sartre|               2|
|Pierre de Ronsard|               2|
+-----------------+----------------+



In [27]:
write.join(Author, ["aid"])\
     .select("name")\
     .groupBy("name")\
     .agg(F.count("name").alias("Nombre de livres"))\
     .show()

+-----------------+----------------+
|             name|Nombre de livres|
+-----------------+----------------+
| Jean Paul Sartre|               2|
|Pierre de Ronsard|               2|
+-----------------+----------------+



**Result:** Les deux auteurs ont écrit le même nombre de livres

### 7- Les personnes qui n’ont pas encore rendu les livres.

In [52]:
# SQL
spark.sql(''' select sname
              from Student as s join borrow as b on s.sid = b.sid
              where return_time is Null                     
              
              ''').show()

+-----+
|sname|
+-----+
| toto|
+-----+



In [53]:
# DSL
Student.join(borrow, on=["sid"])\
     .select("sname")\
     .filter(col("return_time").isNull())\
     .show()

+-----+
|sname|
+-----+
| toto|
+-----+



### 8- Créer une nouvelle colonne dans la table borrow qui prend la valeur 1, si la durée d'emprunt est supérieur à 3 mois,  sinon 0.


In [59]:
#DSL
borrow_csv =borrow.withColumn("checkout-time", F.to_date(col("checkout-time"), "dd-MM-yyyy"))\
                  .withColumn("return-time", F.to_date(col("return-time"), "dd-MM-yyyy"))\
                  .withColumn("Beyond 3 months", F.when(F.datediff(col("return-time"), col("checkout-time")) > 90, 1).otherwise(0))
borrow_csv.show()

+---+----+-------------+-----------+---------------+
|sid| bid|checkout-time|return-time|Beyond 3 months|
+---+----+-------------+-----------+---------------+
|S15|0003|   2020-01-02| 2020-02-01|              0|
|S15|0002|   2020-06-13|       null|              0|
|S15|0001|   2020-06-13| 2020-10-13|              1|
|S16|0002|   2020-01-24| 2020-01-24|              0|
|S17|0001|   2020-04-12| 2020-07-01|              0|
+---+----+-------------+-----------+---------------+



In [61]:
# Exportation de la base borrow_csv en format csv :
borrow_csv.toPandas().to_csv("output/borrow.csv", header=True)

### 9- Les livres qui n’ont jamais été empruntés. 

In [54]:
# SQL
spark.sql(''' select bid, title
              from bookSQL
              EXCEPT
              select distinct(b.bid), title
              from borrowSQL as b join bookSQL as bk on b.bid = bk.bid
             
              ''').show()

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



In [105]:
# DSL
dh1 = book.select("bid", "title")
dh2 = borrow.join(book, borrow.bid == book.bid)\
            .select(book.bid, book.title).dropDuplicates()
dh1.subtract(dh2).show()

+----+----------+
| bid|     title|
+----+----------+
|0004|Les Amours|
+----+----------+



In [48]:
#Fermeture de la session saprk :
spark.stop()